In [5]:
import numpy as np 

import pandas as pd 

import json

from geopy.geocoders import Nominatim 

import requests
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [50]:
url = 'https://en.wikipedia.org/wiki/Districts_of_Warsaw'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

In [51]:
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

In [52]:
df = pd.DataFrame(data, columns=['District','Population','Area'])

In [53]:
df.head()

,District,Population,Area
0,None,None,None
1,Mokotów,"220,682",35.4 km2 (13.7 sq mi)
2,Praga Południe,"178,665",22.4 km2 (8.6 sq mi)
3,Ursynów,"145,938",48.6 km2 (18.8 sq mi)
4,Wola,"137,519",19.26 km2 (7.44 sq mi)


In [54]:
df.dropna(inplace=True)
df

,District,Population,Area
1,Mokotów,"220,682",35.4 km2 (13.7 sq mi)
2,Praga Południe,"178,665",22.4 km2 (8.6 sq mi)
3,Ursynów,"145,938",48.6 km2 (18.8 sq mi)
4,Wola,"137,519",19.26 km2 (7.44 sq mi)
5,Bielany,"132,683",32.3 km2 (12.5 sq mi)
6,Targówek,"123,278",24.37 km2 (9.41 sq mi)
7,Śródmieście,"122,646",15.57 km2 (6.01 sq mi)
8,Bemowo,"115,873",24.95 km2 (9.63 sq mi)
9,Białołęka,"96,588",73.04 km2 (28.20 sq mi)
10,Ochota,"84,990",09.7 km2 (3.7 sq mi)


In [55]:
df.drop(df.index[18], inplace=True)

In [56]:
df.drop(['Population', 'Area'], axis=1, inplace=True)

In [37]:
!pip install geocoder

In [28]:
import geocoder

In [29]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Warsaw, Poland'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [30]:
coords = [ get_latlng(neighborhood) for neighborhood in df["District"].tolist() ]

In [31]:
coords

[[52.19539000000003, 21.008500000000026],
 [52.236330000000066, 21.098400000000026],
 [52.15418000000005, 21.037860000000023],
 [52.23903000000007, 20.971230000000048],
 [52.27697000000006, 20.947780000000023],
 [52.27726000000007, 21.06594000000007],
 [52.235600000000034, 21.010370000000023],
 [52.25269000000003, 20.91244000000006],
 [52.310970000000054, 20.99324000000007],
 [52.21314000000007, 20.970690000000047],
 [52.215050000000076, 21.157580000000053],
 [52.256800000000055, 21.029760000000067],
 [52.19786000000005, 20.891140000000064],
 [52.265720000000044, 20.974830000000054],
 [52.191410000000076, 20.952120000000036],
 [52.16566000000006, 21.086490000000026],
 [52.26579000000004, 21.169940000000054],
 [52.222920000000045, 21.230740000000026],
 [52.235600000000034, 21.010370000000023]]

In [57]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [58]:
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [59]:
print(df.shape)
df

(18, 3)


,District,Latitude,Longitude
1,Mokotów,52.23633,21.09840
2,Praga Południe,52.15418,21.03786
3,Ursynów,52.23903,20.97123
4,Wola,52.27697,20.94778
5,Bielany,52.27726,21.06594
6,Targówek,52.23560,21.01037
7,Śródmieście,52.25269,20.91244
8,Bemowo,52.31097,20.99324
9,Białołęka,52.21314,20.97069
10,Ochota,52.21505,21.15758


In [60]:
df.to_csv("df.csv", index=False)

Map of Warsaw

In [61]:
# get the coordinates of Warsaw
address = 'Warsaw, Poland'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Warsaw, Poland {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Warsaw, Poland 52.2337172, 21.0714111288323.


In [63]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['District']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [64]:
# save the map as HTML file
map_kl.save('map_kl.html')

In [65]:
# The code was removed by Watson Studio for sharing.

In [67]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['District']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [68]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['District', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1229, 7)


,District,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Mokotów,52.23633,21.0984,"Park ""Znicza""",52.237999,21.095642,Park
1,Mokotów,52.23633,21.0984,Bazar Szembeka,52.240648,21.102356,Farmers Market
2,Mokotów,52.23633,21.0984,Zdrofit Gocław,52.235442,21.084058,Gym / Fitness Center
3,Mokotów,52.23633,21.0984,McFIT,52.233696,21.102616,Gym
4,Mokotów,52.23633,21.0984,Park Nad Balatonem,52.228161,21.087832,Park


In [69]:
venues_df.groupby(["District"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
District,,,,,,
Bemowo,23,23,23,23,23,23
Białołęka,100,100,100,100,100,100
Bielany,25,25,25,25,25,25
Mokotów,96,96,96,96,96,96
Ochota,31,31,31,31,31,31
Praga Południe,100,100,100,100,100,100
Praga Północ,53,53,53,53,53,53
Rembertów,14,14,14,14,14,14
Targówek,100,100,100,100,100,100


In [70]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 202 uniques categories.


In [71]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Park', 'Farmers Market', 'Gym / Fitness Center', 'Gym',
       'Restaurant', 'Supermarket', 'Chinese Restaurant', 'Plaza',
       'Beer Bar', 'Coffee Shop', 'Hostel', 'Clothing Store',
       'Water Park', 'Liquor Store', 'Café', 'Italian Restaurant',
       'Recreation Center', 'Dessert Shop', 'Gastropub',
       'Indian Restaurant', 'Sushi Restaurant', 'Multiplex',
       'Burger Joint', 'Gym Pool', 'Grocery Store', 'Shopping Mall',
       'Eastern European Restaurant', 'Bistro', 'Sandwich Place',
       'Convenience Store', 'African Restaurant', 'Hotel', 'Market',
       'Diner', 'Fast Food Restaurant', 'Bookstore', 'Hardware Store',
       'Mexican Restaurant', 'Pharmacy', 'Kebab Restaurant',
       'Historic Site', 'Bus Station', 'Food Court', 'Drugstore',
       'Discount Store', 'Bus Line', 'Electronics Store', 'Lake',
       'Train Station', 'Trail'], dtype=object)

In [72]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

Analyze Each Neighborhood

In [74]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['District'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(1229, 203)


,Neighborhoods,African Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Mokotów,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Mokotów,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Mokotów,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Mokotów,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Mokotów,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(18, 203)


,Neighborhoods,African Restaurant,Airport,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Bemowo,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.043478,...,0.00,0.00,0.00,0.043478,0.000000,0.000000,0.00,0.00,0.00,0.000000
1,Białołęka,0.000000,0.000000,0.020000,0.00,0.01,0.00,0.000000,0.00,0.000000,...,0.01,0.00,0.01,0.010000,0.000000,0.010000,0.01,0.00,0.00,0.000000
2,Bielany,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000
3,Mokotów,0.010417,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.010417,0.000000,0.00,0.00,0.00,0.000000
4,Ochota,0.000000,0.000000,0.032258,0.00,0.00,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000
5,Praga Południe,0.000000,0.000000,0.010000,0.00,0.00,0.01,0.000000,0.00,0.010000,...,0.00,0.01,0.01,0.000000,0.000000,0.000000,0.01,0.00,0.00,0.000000
6,Praga Północ,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.00,0.037736,0.000000,0.000000,0.00,0.00,0.00,0.000000
7,Rembertów,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000
8,Targówek,0.000000,0.000000,0.000000,0.00,0.00,0.01,0.000000,0.01,0.010000,...,0.00,0.00,0.01,0.000000,0.000000,0.010000,0.00,0.01,0.00,0.000000
9,Ursus,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.01,0.020000,...,0.00,0.00,0.02,0.010000,0.000000,0.020000,0.00,0.00,0.00,0.000000


In [76]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

7

In [77]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [78]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Bemowo,0.043478
1,Białołęka,0.000000
2,Bielany,0.000000
3,Mokotów,0.010417
4,Ochota,0.032258


Cluster Neighborhoods

In [79]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 2, 1, 0, 1, 0, 0, 2], dtype=int32)

In [80]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [81]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Bemowo,0.043478,1
1,Białołęka,0.000000,0
2,Bielany,0.000000,0
3,Mokotów,0.010417,2
4,Ochota,0.032258,1


In [83]:
# merge 
kl_merged = kl_merged.join(df.set_index("District"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(18, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Bemowo,0.043478,1,52.31097,20.99324
1,Białołęka,0.000000,0,52.21314,20.97069
2,Bielany,0.000000,0,52.27726,21.06594
3,Mokotów,0.010417,2,52.23633,21.09840
4,Ochota,0.032258,1,52.21505,21.15758


In [84]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(18, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
8,Targówek,0.000000,0,52.23560,21.01037
14,Wola,0.000000,0,52.27697,20.94778
13,Wilanów,0.000000,0,52.26579,21.16994
12,Wesoła,0.000000,0,52.23560,21.01037
11,Wawer,0.000000,0,52.25680,21.02976
10,Ursynów,0.000000,0,52.23903,20.97123
7,Rembertów,0.000000,0,52.22292,21.23074
17,Żoliborz,0.000000,0,52.19141,20.95212
5,Praga Południe,0.000000,0,52.15418,21.03786
2,Bielany,0.000000,0,52.27726,21.06594


In [85]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [86]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

Examine Clusters

In [87]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
8,Targówek,0.0,0,52.23560,21.01037
14,Wola,0.0,0,52.27697,20.94778
13,Wilanów,0.0,0,52.26579,21.16994
12,Wesoła,0.0,0,52.23560,21.01037
11,Wawer,0.0,0,52.25680,21.02976
10,Ursynów,0.0,0,52.23903,20.97123
7,Rembertów,0.0,0,52.22292,21.23074
17,Żoliborz,0.0,0,52.19141,20.95212
5,Praga Południe,0.0,0,52.15418,21.03786
2,Bielany,0.0,0,52.27726,21.06594


In [88]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
16,Śródmieście,0.033898,1,52.25269,20.91244
4,Ochota,0.032258,1,52.21505,21.15758
6,Praga Północ,0.037736,1,52.19786,20.89114
0,Bemowo,0.043478,1,52.31097,20.99324


In [89]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
9,Ursus,0.010000,2,52.26572,20.97483
3,Mokotów,0.010417,2,52.23633,21.09840
15,Włochy,0.012048,2,52.16566,21.08649


Conclusion

With the results obtained we can see that most of the malls area located in cluster 1, while cluster 2 has less malls and cluster 0 has no shopping malls.
This means that the districts located in cluster 1 have more competition than in other clusters.
This project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition and property developers with unique selling propositions can also open new shopping malls in neighborhoods in cluster 2 with moderate competition.